<a href="https://colab.research.google.com/github/vldzmtnsdl/ESAA-/blob/ESAA/0927_%EC%84%B8%EC%85%98%ED%95%84%EC%82%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### [NLP 언제까지 미룰래? 일단 들어와!!]4. word Embedding

In [53]:
import pandas as pd
import numpy as np

In [54]:
train=pd.read_csv('/content/drive/MyDrive/train.csv')

In [55]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def text2sequence(train_text, max_len=100):
    
    tokenizer = Tokenizer() #keras의 vectorizing 함수 호출
    tokenizer.fit_on_texts(train_text) #train 문장에 fit
    train_X_seq = tokenizer.texts_to_sequences(train_text) #각 토큰들에 정수 부여
    vocab_size = len(tokenizer.word_index) + 1 #모델에 알려줄 vocabulary의 크기 계산
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len) #설정한 문장의 최대 길이만큼 padding
    
    return X_train, vocab_size, tokenizer

train_X, vocab_size, vectorizer = text2sequence(train['text'], max_len = 100)

vocab_size :  42331


1.keras embedding layer

In [56]:
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.layers import Embedding


In [57]:
max_len=100

In [58]:
model=Sequential()
model.add(Embedding(vocab_size,128,input_length=max_len))

2. word2vec

In [59]:
import gensim
word2vec = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz', binary = True)

In [60]:
embedding_matrix=np.zeros((vocab_size,300))

for index,word in enumerate(tokenizer.word_index):
  if word in word2vec:
    embedding_vector=word2vec[word]
    embedding_matrix[index]=embedding_vector
  else:
    print('word2vec에 없는 단어입니다.')
    break

In [61]:
model=Sequential()
model.add(Embedding(vocab_size,300,weights=[embedding_matrix],input_length=max_len))

3.glove

In [62]:
glove=dict()
f=open('/content/drive/MyDrive/glove.txt')
for line in f:
    values = line.split()
    word = values[0]
    vector = np.array(values[1:], dtype='float32')
    glove[word] = vector
f.close()

In [63]:
tokenizer = Tokenizer()

embedding_matrix = np.zeros((vocab_size, 300)) 

for index, word in enumerate(tokenizer.word_index): 
    if word in glove: 
        embedding_vector = glove[word] 
        embedding_mxtrix[i] = embedding_vector
    else:
        print("glove 없는 단어입니다.")
        break

In [64]:
model = Sequential()
model.add(Embedding(vocab_size, 300,weights = [embedding_matrix], input_length = max_len))

4. fasttext

In [65]:
from gensim.models.keyedvectors import KeyedVectors
FastText = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/fasttext.vec', binary=True, unicode_errors='ignore')


In [66]:
embedding_matrix = np.zeros((vocab_size, 300))
for index, word in enumerate(tokenizer.word_index): 
    if word in word2vec: 
        embedding_vector = word2vec[word] 
        embedding_mxtrix[i] = embedding_vector 
model = Sequential()
model.add(Embedding(vocab_size, 300,weights = [embedding_matrix], input_length = max_len))

## [NLP 언제까지 미룰래? 일단 들어와!!] 5. 대회적용

In [67]:
import tqdm as tqdm


In [70]:
pip install konlpy


     |████████████████████████████████| 19.4 MB 22.3 MB/s 
     |████████████████████████████████| 448 kB 55.9 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


1. 데이터 전처리

In [71]:
from konlpy.tag import Okt
import re
import tqdm 

def text_preprocessing(text_list):
    
    stopwords = ['을', '를', '이', '가', '은', '는', 'null'] #불용어 설정
    tokenizer = Okt() #형태소 분석기 
    token_list = []
    
    for text in tqdm.tqdm(text_list):
        txt = re.sub('[^가-힣a-z]', ' ', text) #한글과 영어 소문자만 남기고 다른 글자 모두 제거
        token = tokenizer.morphs(txt) #형태소 분석
        token = [t for t in token if t not in stopwords or type(t) != float] #형태소 분석 결과 중 stopwords에 해당하지 않는 것만 추출
        token_list.append(token)
        
    return token_list, tokenizer
    
train['token'], okt = text_preprocessing(train['text'])

100%|██████████| 54879/54879 [01:27<00:00, 623.97it/s]


2. vectorization

In [73]:
def text2sequence(train_text,max_len=1000):
  tokenizer=Tokenizer()
  tokenizer.fit_on_texts(train_text)
  train_X_seq=tokenizer.texts_to_sequences(train_text)
  vocab_size=len(tokenizer.word_index)+1
  print('vocab_size:',vocab_size)
  X_train=pad_sequences(train_X_seq,maxlen=max_len)
  return X_train,vocab_size,tokenizer

train_y=train['author']
train_X,vocab_szie,vectorizer=text2sequence(train['token'],max_len=100)
print(train_X.shape,train_y.shape)

vocab_size: 36342
(54879, 100) (54879,)


3.Embedding

In [76]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz', binary = True)
embedding_matrix = np.zeros((vocab_size, 300))

for index, word in enumerate(tokenizer.word_index):
    if word in word2vec:
        embedding_vector = word2vec[word] 
        embedding_matrix[index] = embedding_vector 
    else:
        print("word2vec에 없는 단어입니다.")
        break

4. modeling

In [77]:
def LSTM(vocab_size, max_len=1000):
    model = Sequential()
    model.add(Embedding(vocab_size, 300,weights = [embedding_matrx], input_length = max_len)) 
    model.add(SpatialDropout1D(0.3))
    model.add(LSTM(64))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu', kernel_regularizer = regularizers.l2(0.001)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
    model.summary()
    return mode